##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 TFX 和 Vertex Pipelines 从 BigQuery 读取数据


<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://tensorflow.google.cn/tfx/tutorials/tfx/gcp/vertex_pipelines_bq"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 Tensorflow.org 上查看</a> </td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/gcp/vertex_pipelines_bq.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 运行</a> </td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/gcp/vertex_pipelines_bq.ipynb"><img width="32px" src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/gcp/vertex_pipelines_bq.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a></td>
<td><a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?q=download_url%3Dhttps://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tfx/tutorials/tfx/gcp/vertex_pipelines_bq.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">在 Google Cloud Vertex AI Workbench 中运行</a></td>
</table></div>


这个基于笔记本的教程将使用 [Google Cloud BigQuery](https://cloud.google.com/bigquery) 作为数据源来训练 ML 模型。ML 流水线将使用 TFX 构建并在 Google Cloud Vertex Pipelines 上运行。

此笔记本基于我们在[适用于 Vertex Pipelines 的简单 TFX 流水线教程](https://tensorflow.google.cn/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)中构建的 TFX 流水线。如果您还未阅读该教程，应在继续使用此笔记本之前阅读它。

[BigQuery](https://cloud.google.com/bigquery) 是专为实现业务敏捷性而设计的无服务器、高度可扩展且经济高效的多云数据仓库。TFX 可用于从 BigQuery 读取训练数据并向 BigQuery [发布经过训练的模型](https://tensorflow.google.cn/tfx/api_docs/python/tfx/extensions/google_cloud_big_query/pusher/executor/Executor)。

在本教程中，我们将使用 `BigQueryExampleGen` 组件，该组件将数据从 BigQuery 读取到 TFX 流水线。


此笔记本旨在于 [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb) 或 [AI Platform Notebooks](https://cloud.google.com/ai-platform-notebooks) 上运行。如果您不使用其中之一，只需点击上面的“在 Google Colab 中运行”按钮即可。

## 设置

如果您已经完成[适用于 Vertex Pipelines 的简单 TFX 流水线教程](https://tensorflow.google.cn/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)，您将拥有一个 GCP 工作项目和一个 GCS 存储桶，这就是本教程所需要的全部内容。如果您错过了初步教程，请先阅读它。

### 安装 python 软件包

我们将安装所需的 Python 软件包（包括 TFX 和 KFP），以编写 ML 流水线并将作业提交到 Vertex Pipelines。

In [ ]:
# Use the latest version of pip.
!pip install --upgrade pip
!pip install --upgrade "tfx[kfp]<2"

### 卸载 Shapely

TODO(b/263441833) 这是避免 ImportError 的临时解决方案。最终，应该通过支持最新版本的 Bigquery 来处理，而不是卸载其他额外的依赖项。

In [ ]:
!pip uninstall shapely -y

#### 是否已重新启动运行时？

如果您使用的是 Google Colab，则在首次运行上面的代码单元时必须重新启动运行时，方法是点击上面的“重新启动运行时”按钮或使用“运行时 &gt; 重新启动运行时...”菜单。这样做的原因是 Colab 加载软件包的方式。

如果您不在 Colab 上，可以使用以下代码单元重新启动运行时。

In [ ]:
# docs_infra: no_execute
import sys
if not 'google.colab' in sys.modules:
  # Automatically restart kernel after installs
  import IPython
  app = IPython.Application.instance()
  app.kernel.do_shutdown(True)

### 登录 Google 获取此笔记本

如果您在 Colab 上运行此笔记本，请使用您的用户帐户进行身份验证：

In [ ]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

**如果您在 AI Platform Notebooks 上**，请在运行下一部分之前通过 Google Cloud 进行身份验证，方法是运行

```sh
gcloud auth login
```

运行位置为**终端窗口** （可通过菜单中的**文件** &gt; **新建**打开）。您只需对每个笔记本实例执行一次此操作。

检查软件包版本。

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import kfp
print('KFP version: {}'.format(kfp.__version__))

### 设置变量

我们将在下面设置一些用于自定义流水线的变量。所需信息如下：

- GCP 项目 ID 和编号。请参阅[找出项目 ID 和编号](https://cloud.google.com/resource-manager/docs/creating-managing-projects#identifying_projects)。
- 用于运行流水线的 GCP 区域。有关 Vertex Pipelines 在其中可用的区域的更多信息，请参阅 [Vertex AI 位置指南](https://cloud.google.com/vertex-ai/docs/general/locations#feature-availability)。
- 用于存储流水线输出的 Google Cloud Storage 存储桶。

**在下面的代码单元中输入所需值，然后再运行它**。


In [ ]:
GOOGLE_CLOUD_PROJECT = ''         # <--- ENTER THIS
GOOGLE_CLOUD_PROJECT_NUMBER = ''  # <--- ENTER THIS
GOOGLE_CLOUD_REGION = ''          # <--- ENTER THIS
GCS_BUCKET_NAME = ''              # <--- ENTER THIS

if not (GOOGLE_CLOUD_PROJECT and  GOOGLE_CLOUD_PROJECT_NUMBER and GOOGLE_CLOUD_REGION and GCS_BUCKET_NAME):
    from absl import logging
    logging.error('Please set all required parameters.')

设置 `gcloud` 以使用您的项目。

In [ ]:
!gcloud config set project {GOOGLE_CLOUD_PROJECT}

In [ ]:
PIPELINE_NAME = 'penguin-bigquery'

# Path to various pipeline artifact.
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' Python module.
MODULE_ROOT = 'gs://{}/pipeline_module/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' data.
DATA_ROOT = 'gs://{}/data/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# This is the path where your model will be pushed for serving.
SERVING_MODEL_DIR = 'gs://{}/serving_model/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))

默认情况下，Vertex Pipelines 使用格式为 `[project-number]-compute@developer.gserviceaccount.com` 的默认 GCE VM 服务帐户。我们需要授予此帐户使用 BigQuery 的权限，以便在流水线中访问 BigQuery。我们将向帐户中添加“BigQuery 用户”角色。

In [ ]:
!gcloud projects add-iam-policy-binding {GOOGLE_CLOUD_PROJECT} \
  --member=serviceAccount:{GOOGLE_CLOUD_PROJECT_NUMBER}-compute@developer.gserviceaccount.com \
  --role=roles/bigquery.user

有关服务帐户和 IAM 配置的更多信息，请参阅 [Vertex 文档](https://cloud.google.com/vertex-ai/docs/pipelines/configure-project)。

## 创建流水线

TFX 流水线是使用 Python API 定义的，就像我们在[适用于 Vertex Pipelines 的简单 TFX 流水线教程](https://tensorflow.google.cn/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)中所做的那样。我们之前使用的是 `CsvExampleGen`，其从 CSV 文件中读取数据。在本教程中，我们将使用 [`BigQueryExampleGen`](https://tensorflow.google.cn/tfx/api_docs/python/tfx/v1/extensions/google_cloud_big_query/BigQueryExampleGen) 组件，该组件从 BigQuery 中读取数据。


### 准备 BigQuery 查询

我们将使用相同的 [Palmer Penguins 数据集](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)。但是，我们将从使用相同 CSV 文件填充的 BigQuery 表 `tfx-oss-public.palmer_penguins.palmer_penguins` 中读取它。

如果您使用的是 Google Colab，则可以直接检查 BigQuery 表的内容。

In [ ]:
# docs_infra: no_execute
%%bigquery --project {GOOGLE_CLOUD_PROJECT}
SELECT *
FROM `tfx-oss-public.palmer_penguins.palmer_penguins`
LIMIT 5

除了作为标签的 `species` 外，所有特征均已标准化为 0~1。我们将建立一个预测企鹅 `species` 的分类模型。

`BigQueryExampleGen` 需要一个查询来指定要获取的数据。因为我们将使用表中所有行的所有字段，所以查询非常简单。根据 <a>BigQuery 标准 SQL 语法</a>，您还可以按需指定字段名称并添加 <code>WHERE</code> 条件。

In [ ]:
QUERY = "SELECT * FROM `tfx-oss-public.palmer_penguins.palmer_penguins`"

### 编写模型代码。

我们将使用与[简单 TFX 流水线教程](https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_simple)中相同的模型代码。

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

# Copied from https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_simple

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _make_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://tensorflow.google.cn/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _make_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

将模块文件复制到可从流水线组件进行访问的 GCS。因为模型训练发生在 GCP 上，所以我们需要上传此模型定义。

否则，您可能需要构建包含模块文件的容器映像并使用该映像来运行流水线。

In [ ]:
!gsutil cp {_trainer_module_file} {MODULE_ROOT}/

### 编写流水线定义

我们将定义一个函数来创建 TFX 流水线。我们需要使用 `BigQueryExampleGen`，其采用 `query` 作为实参。与上一教程变化较大的是我们需要传递 `beam_pipeline_args`，它在组件执行之时传递给组件。我们将使用 `beam_pipeline_args` 向 BigQuery 传递其他形参。


In [ ]:
from typing import List, Optional

def _create_pipeline(pipeline_name: str, pipeline_root: str, query: str,
                     module_file: str, serving_model_dir: str,
                     beam_pipeline_args: Optional[List[str]],
                     ) -> tfx.dsl.Pipeline:
  """Creates a TFX pipeline using BigQuery."""

  # NEW: Query data in BigQuery as a data source.
  example_gen = tfx.extensions.google_cloud_big_query.BigQueryExampleGen(
      query=query)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a file destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=components,
      # NEW: `beam_pipeline_args` is required to use BigQueryExampleGen.
      beam_pipeline_args=beam_pipeline_args)

## 在 Vertex Pipelines 上运行流水线。

我们将使用 Vertex Pipelines 来运行流水线，就像我们在[适用于 Vertex Pipelines 的简单 TFX 流水线教程](https://tensorflow.google.cn/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)中所做的那样。


我们还需要为 BigQueryExampleGen 传递 `beam_pipeline_args`。它包括 GCP 项目的名称和 BigQuery 执行的临时存储等配置。

In [ ]:
# docs_infra: no_execute
import os

# We need to pass some GCP related configs to BigQuery. This is currently done
# using `beam_pipeline_args` parameter.
BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS = [
   '--project=' + GOOGLE_CLOUD_PROJECT,
   '--temp_location=' + os.path.join('gs://', GCS_BUCKET_NAME, 'tmp'),
   ]

PIPELINE_DEFINITION_FILE = PIPELINE_NAME + '_pipeline.json'

runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(),
    output_filename=PIPELINE_DEFINITION_FILE)
_ = runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        query=QUERY,
        module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        serving_model_dir=SERVING_MODEL_DIR,
        beam_pipeline_args=BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS))

生成的定义文件可以使用 kfp 客户端提交。

In [ ]:
# docs_infra: no_execute
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
import logging
logging.getLogger().setLevel(logging.INFO)

aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_REGION)

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.submit()

现在，您可以访问上面输出中的链接或访问 [Google Cloud Console](https://console.cloud.google.com/) 中的“Vertex AI &gt; 流水线”来查看进度。